In [1]:
from itertools import chain

import altair as alt
import pandas as pd
import tensorflow_hub as hub
from hdbscan import HDBSCAN
from IPython.display import display
from umap import UMAP

In [2]:
df = (
    pd.read_csv("cleaned_hm.csv.xz")
    .dropna(subset=["ground_truth_category", "cleaned_hm"], how="any")
    .loc[:, ["cleaned_hm", "ground_truth_category"]]
    .reset_index(drop=True)
)



In [3]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [4]:
docs = df["cleaned_hm"].tolist()
embeddings = embed(docs)

In [5]:
umap = UMAP(min_dist=0.00, n_neighbors=30, metric="cosine", random_state=1234)
embeddings_umap = umap.fit_transform(embeddings)

In [6]:
embeddings_df = pd.DataFrame()
embeddings_df["Document"] = docs
embeddings_df["Component 1"] = embeddings_umap[:, 0]
embeddings_df["Component 2"] = embeddings_umap[:, 1]
embeddings_df["Ground Truth"] = df["ground_truth_category"]

In [7]:
hdbscan = HDBSCAN(cluster_selection_method="leaf", min_cluster_size=len(embeddings_df) // 100)

In [8]:
clusters = hdbscan.fit_predict(embeddings_umap)
embeddings_df["Cluster"] = clusters

In [9]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [10]:
chart = (
    (
        alt.Chart(
            embeddings_df,
            height=1000,
            width=1000,
            title="Happy Moments - USE → UMAP → HDBSCAN",
        )
        .mark_point()
        .encode(
            x=alt.X("Component 1", axis=None),
            y=alt.Y("Component 2", axis=None),
            tooltip=["Document", "Cluster", "Ground Truth"],
            color="Cluster:N",
            shape="Ground Truth"
        )
    )
    .configure_axis(grid=False)
    .configure_view(strokeWidth=0)
    .interactive()
)

In [11]:
chart.save("docs/nlp-use-umap-hdbscan-chart.html")

In [12]:
exemplars = list(map(tuple, chain.from_iterable(hdbscan.exemplars_)))

def exemplar(row):
    return (row["Component 1"], row["Component 2"]) in exemplars

embeddings_df["Exemplar"] = embeddings_df.apply(exemplar, axis=1)


In [13]:
embeddings_df["Exemplar"].value_counts()

False    10829
True      3296
Name: Exemplar, dtype: int64

In [14]:
grouped_df = embeddings_df.groupby("Cluster")

In [15]:
pd.set_option("display.max_colwidth", 200)

In [18]:
for key, item in grouped_df:
    n =  len(grouped_df.get_group(key))
    if key == -1:
        print(f"No Cluster (outlier) \tN Total: {n} ({(n/len(embeddings_df)) * 100:.1f}%)")
        print("-----" * 5)
        continue
    print(f"Cluster: {key} \tN Total: {n} ({(n/len(embeddings_df)) * 100:.1f}%)")
    print("Cluster Exemplars: ")
    display(grouped_df.get_group(key).query("Exemplar")["Document"].to_frame())
    display(grouped_df.get_group(key)["Ground Truth"].value_counts().to_frame())
    print("-----" * 5)

No Cluster (outlier) 	N Total: 6831 (48.4%)
-------------------------
Cluster: 0 	N Total: 144 (1.0%)
Cluster Exemplars: 


,Document
606,I WENT TO MOVIE
749,I WEND TO MOVIE
750,I WENT TO MOVIE
875,I WENT TO MOVIE
877,I WENT TO MOVIE
...,...
13909,I WENT TO MOVIE
13992,I WENT MOVIE
13998,I WENT TO MOVIE
14069,I WENT TO MOVIE


,Ground Truth
leisure,144


-------------------------
Cluster: 1 	N Total: 153 (1.1%)
Cluster Exemplars: 


,Document
94,My cat snuggled with me while we slept.
241,My cat cuddled with me when I went to sleep.
256,I was able to take my cat out for a walk while the sky was clear.
257,"I played with my new kitten, we have only had him for 3 weeks!"
272,My friend sent me pictures of his cat.
...,...
13655,We found a stray cat and couldnt catch it but we fed it . that made me happy .
13666,My wife found a kitten on her morning walk. She is really cute and my kids love her.
13754,"My cats came and cuddled with me, including the male the cat that does not like the cuddle much."
13791,My son learned to meow like a cat.


...

,Ground Truth
affection,105
bonding,36
enjoy_the_moment,18
achievement,2
leisure,2


-------------------------
Cluster: 22 	N Total: 447 (3.2%)
Cluster Exemplars: 


,Document
64,i met my old friends after 5 years.we all had a talk for half a day.we played football like my old days.
93,I saw some friends that I had not seen in a while and it was good to reconnect.
300,I spent some time with a close friend.
354,I met with my school friends and enjoyed a lot.
355,I got to see one of my friends that I havent seen in a long time and we got to catch up.
...,...
13718,Today I meet my childhood friends.
13745,Visit my best friend
13771,A friend called me and we caught up on a lot of things.
13839,I got to see my best friend for the first time in over a month. I've really missed her - like missing a part of myself (my better half practically.)


,Ground Truth
bonding,415
affection,12
achievement,9
enjoy_the_moment,7
leisure,4


-------------------------
